# AI-Powered Anomaly Detection PoC for Subledger Transactions

##**Purpose of this Notebook**

This notebook demonstrates an end-to-end Proof of Concept (PoC) for applying Machine Learning (ML) and Large Language Models (LLMs) to detect and explain anomalies in a subledger accounting dataset.

The workflow showcases how an accounting or finance platform can:

1. Ingest transaction data
2. Engineer meaningful features (dates, accounts, user behaviour, amounts)
3. Apply an IsolationForest model to detect unusual transactions
4. Compute statistical baselines (per account, per vendor, etc.) to give context about “normal” behaviour
5. Use OpenAI models to generate clear, domain-specific explanations for each anomaly
6. Produce enriched output suitable for dashboards, audit workflows, or finance review queues

---

##**What this PoC demonstrates**

- A scalable approach for automated anomaly detection
- How ML models and LLMs can work together:
- ML flags anomalies
- LLM explains why, using statistical context
- A simple, portable pipeline that an engineering team can later convert into production services
- How to ground LLM reasoning in real data to avoid hallucinations

---

##**Who this is for**

- The Product and Engineering team evaluating AI use cases and costs
- Engineering teams exploring how ML + LLMs can enhance auditability
- Finance SMEs who want to understand how AI can surface insights in everyday workflows
- Stakeholders who want a clear, transparent demonstration—not a black box

---

##**What this is NOT**

- Not a fully tuned production model
- Not a fully secure, optimized, or scalable architecture
- Not a replacement for human review
- Not tied to any specific client implementation as it uses fabricated data

This is a transparent, educational PoC that shows how AI can be integrated into real workflows.

---

##**Next Steps**

- Fine tune and understand IF better
- Add in a rule layer, potentially flagging things that IF wouldn't. For example large transactions might not be flagged if in keeping with other regular patterns. This might mean IF doesn't pick it up but it should be flagged.

---

##**High-Level Architecture Overview**

At a high level, this PoC follows this flow:

- Synthetic or real subledger data
  - Input: transaction records (date, amount, accounts, user, vendor, type…)
  - Source: CSV file (transactions.csv)
- Feature engineering
  - Derive extra fields: hour of day, day of week, month, log-transformed amount.
  - Build a feature matrix suitable for ML models.
- Statistical profiling
  - Compute per-account and per-vendor statistics:
    - Average / median amounts
    - Transaction counts
    - Z-scores vs typical amounts
  - These provide ground truth “typical behaviour” for later explanations.
- Anomaly detection (IsolationForest)
  - Train an IsolationForest model on the engineered features.
  - Score each transaction for how unusual it is.
  - Select the top N most anomalous transactions.
- LLM-powered explanations (OpenAI)
  - For each anomalous transaction, send:
    - The raw transaction fields
    - Anomaly score
    - Statistical context (per-account/per-vendor stats)
  - The LLM returns:
    - Severity (low / medium / high)
    - Category (e.g. amount_anomaly, timing_anomaly…)
    - Human-readable explanation
    - Suggested next action for a finance user
- Enriched anomaly output
  - Combine ML output + LLM output into a single DataFrame.
  - Export as CSV for:
    - Internal review
    - Product design discussions
    - Stakeholder demos

Visually, you can think of it as:

```
CSV → Features & Stats → IsolationForest → Top Anomalies → OpenAI Explanations → Enriched Output
```

---


##**Why IsolationForest for This PoC**

IsolationForest is a strong match for anomaly detection in financial and accounting data, especially at the PoC stage. It was selected for several reasons:

1. Purpose-built for anomaly detection

IsolationForest is specifically designed to detect rare, unusual, or outlier behaviours, which is exactly what we want when reviewing subledger transactions.

It works by:

- Randomly “splitting” the data many times
- Checking how easily a data point can be isolated
- Outliers are isolated quickly → scored as anomalous

This works well for transactions whose amounts, timing, or account combinations differ materially from typical patterns.

2. No labels required (unsupervised learning)

    In accounting systems, true anomalies are:

    - Rare
    - Expensive to label
    - Often subjective

    IsolationForest requires no ground-truth labels, making it ideal for:

    - POCs
    - Legacy datasets without classifications
    - Early stages of an anomaly detection project

3. Handles high-dimensional + mixed data

   Subledger transactions have a mix of:
    - Numeric fields (amount, hour, month)
    - Categorical fields (account, vendor, user, type)

    IsolationForest works well when combined with:

    - One-hot encoding for categorical variables
    - Numeric transformations (log amounts)

This gives it the flexibility needed for financial data structures.

4. Fast, lightweight, and scalable

IsolationForest:

- Trains quickly (even on large datasets)
- Is memory-efficient
- Offers predictable inference performance

This makes it suitable as a backend microservice in a SaaS product.

5. Works well with LLM explanations

IsolationForest provides:

- An anomaly score
- A binary anomaly flag

But it doesn’t explain why a transaction is unusual.
This makes it a perfect pairing with LLMs:

- IsolationForest identifies suspicious transactions
- The LLM interprets and explains them using statistical context

This combination provides:

- Transparency
- Auditability
- User trust
- High-value insights for finance teams

6. A simple, transparent model for stakeholders

Since this is a PoC, we wanted a model that:
- Stakeholders can understand
- Isn’t “too magical”
- Requires minimal ML expertise to explain

IsolationForest fits that perfectly.

Here is a video explaining IsolationForest in more detail and visuals

[IsolationForest](https://www.youtube.com/watch?v=kN--TRv1UDY

---

##**How to Run This Notebook**

To run this PoC end-to-end:

If the notebook has been shared via Google,

1. Open in Google Colab (if the link has not been shared with you)
   - Upload the notebook to Colab or open it from GitHub/Drive.

2. Set your OpenAI API key
   - In Colab, go to: Left sidebar → Key Icon
   - Add a new variable:
     - Enable Notebook Access
     - Name: OPENAI_API_KEY
     - Value: your OpenAI secret key (sk-...)

3. Run all cells in order
   - Use: Runtime → Run all
   - Or step through each cell from top to bottom.
   - The notebook will:
     - Generate or load transaction data
     - Build features and stats
     - Train IsolationForest
     - Call OpenAI to explain anomalies
     - Produce an enriched anomalies table
   - Inspect the results
     - Look at the df_enriched.head() outputs.
     - Check the CSV file (e.g. enriched_anomalies_openai.csv) in the Colab file browser. (it'll be in the `Content` folder)
     - Optionally download it for further analysis or inclusion in slides.
   - Modify configuration as needed
     - Adjust TOP_N (number of anomalies to explain).
     - Tweak the IsolationForest contamination rate.
     - Edit the system prompt to refine explanation style or categories.


I'll stop waffling now and you can get to testing!!!

**Step 1 — Install Required Python Packages**

This step ensures that all necessary Python libraries are available in the Colab environment.

- Installs pandas - Used for loading, cleaning, merging, and analysing transaction data.

- Installs scikit-learn - Provides the IsolationForest anomaly detection model and preprocessing utilities (OneHotEncoder, Pipelines, etc.).

- Updates the openai client SDK to the latest version. Required for calling OpenAI’s LLMs to generate explanations for anomalies. This ensures we have the latest features such as structured output (response_format={"type":"json_object"}).

Because Google Colab resets packages each session, this cell is required every time you open the notebook.

In [74]:
!pip install pandas scikit-learn
!pip install --upgrade openai

**Step 2 — Import Required Libraries**

In this step we import the core libraries used throughout the notebook.

What they’re for:

- pandas, numpy – work with tabular data and numbers.
- IsolationForest – the anomaly detection model.
- ColumnTransformer, OneHotEncoder, Pipeline – build a preprocessing + modelling pipeline that:
  - keeps numeric features as-is
  - one-hot encodes categorical features
  - runs everything through IsolationForest.
- json – formats transactions to send to the LLM and parses JSON responses.

In [75]:
import pandas as pd
import numpy as np

from sklearn.ensemble import IsolationForest
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

import json

**Step 3 — Configure OpenAI Client**

This step sets up access to the OpenAI API so we can generate explanations for anomalous transactions.

What’s happening:

- Load API key from Colab’s secure storage
The key should be added in Colab → Settings → Variables under OPENAI_API_KEY.
- Initialize the OpenAI client
This lets the notebook send requests to the OpenAI models.
- Select the model
gpt-4.1-mini is fast, inexpensive, and strong enough for structured anomaly explanations.

In [76]:
from google.colab import userdata
from openai import OpenAI

api_key = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

MODEL_NAME = "gpt-4.1-mini"  # good default for this use case

**Step 4 — Generate Synthetic Transaction Data with Built-In Anomalies**

This step creates a fake subledger dataset so the PoC can run even without real customer data. The data is shaped to look realistic and includes some deliberately unusual transactions for the model to find.

**What the code does:**

1. Sets up randomness and dataset size
   - rng = np.random.default_rng(42) makes results reproducible.
   - n_rows = 5000 controls how many transactions we simulate.
2. Builds realistic base data
   - Date range over the last 180 days, mostly in business hours (08:00–20:00).
   - Random currencies, accounts, users, vendors, and transaction types.
   - Transaction amounts drawn from a log-normal distribution (skewed like real financial data), with mostly positive and some negative amounts.
3. Creates a DataFrame df
   - Columns: transaction_id, date_time, amount, currency, debit_account, credit_account, user_id, vendor, transaction_type.
4. Injects clear anomalies
   - Very large amounts for some rows (50k–250k).
   - Weird account combinations using 9999-Miscellaneous / 8888-Suspense.
   - Night-time postings between 00:00–04:00.
   - Marks these rows with is_synthetic_anomaly = True so we can later check if the model finds them.
5. Quick sanity check
   - df.head() shows a sample of rows.
   - df["is_synthetic_anomaly"].value_counts() shows how many anomalies were injected.

**Things you can tweak:**

- n_rows – scale up/down the size of the dataset.
- The account lists, vendors, and currencies – to better match our use cases domain.
- n_anomalies - scale up/down the number of anomalies you want to inject.
- The anomaly patterns:
  - Range of “very large” amounts.
  - How many anomalies: n_anomalies.
  - Definitions of “weird” accounts or times.
- The probabilities for transaction_types to change the mix of invoices/payments/journals/adjustments.

These knobs let you simulate different risk profiles or transaction behaviours for experimentation.

In [77]:
rng = np.random.default_rng(42)

n_rows = 5000

# 1) Date range: last 180 days
date_range = pd.date_range(end=pd.Timestamp.today(), periods=180, freq="D")
dates = rng.choice(date_range, size=n_rows)

# 2) Currencies, accounts, users, vendors, transaction types
currencies = ["USD", "EUR", "GBP"]
debit_accounts = [
    "6000-Marketing",
    "6100-Travel",
    "6200-IT",
    "6300-ProfessionalFees",
    "6400-Rent",
    "6500-Salaries",
]
credit_accounts = [
    "1000-Cash",
    "2000-AP",
    "2100-AccruedExpenses",
    "2200-DeferredRevenue",
]
users = [f"user_{i}" for i in range(1, 16)]
vendors = [
    "ACME Events",
    "Globex IT",
    "Initech Consulting",
    "Stark Supplies",
    "Wayne Travel",
    "Umbrella Services",
]
transaction_types = ["invoice", "payment", "journal", "adjustment"]

# 3) Amounts: mostly "normal" with some randomness
# Use log-normal-like distribution for realistic skew
base_amounts = rng.lognormal(mean=7, sigma=0.5, size=n_rows)  # around a few thousand
signs = rng.choice([1, -1], size=n_rows, p=[0.8, 0.2])  # mostly positive, some negative
amounts = np.round(base_amounts * signs, 2)

# 4) Assemble base dataframe
df = pd.DataFrame({
    "transaction_id": [f"tx_{i}" for i in range(1, n_rows + 1)],
    "date_time": dates + pd.to_timedelta(rng.integers(8, 20, size=n_rows), unit="h"),  # business hours 08:00–20:00
    "amount": amounts,
    "currency": rng.choice(currencies, size=n_rows),
    "debit_account": rng.choice(debit_accounts, size=n_rows),
    "credit_account": rng.choice(credit_accounts, size=n_rows),
    "user_id": rng.choice(users, size=n_rows),
    "vendor": rng.choice(vendors, size=n_rows),
    "transaction_type": rng.choice(transaction_types, size=n_rows, p=[0.4, 0.3, 0.2, 0.1]),
})

# 5) Inject some "obvious" anomalies so we can see if the model finds them
n_anomalies = 40
anomaly_indices = rng.choice(df.index, size=n_anomalies, replace=False)

# a) Very large amounts
large_idx = anomaly_indices[:15]
df.loc[large_idx, "amount"] = np.round(rng.uniform(50000, 250000, size=len(large_idx)), 2)

# b) Weird account combinations (e.g., rent against cash, salaries against deferred revenue)
weird_idx = anomaly_indices[15:30]
df.loc[weird_idx, "debit_account"] = rng.choice(
    ["9999-Miscellaneous", "8888-Suspense"], size=len(weird_idx)
)
df.loc[weird_idx, "credit_account"] = rng.choice(
    ["9999-Miscellaneous", "8888-Suspense"], size=len(weird_idx)
)

# c) Odd posting times (middle of the night)
night_idx = anomaly_indices[30:]
df.loc[night_idx, "date_time"] = df.loc[night_idx, "date_time"].dt.normalize() + pd.to_timedelta(
    rng.integers(0, 5, size=len(night_idx)), unit="h"
)  # 00:00–04:00

# Mark which ones we deliberately made anomalous (for our own evaluation, not for the model)
df["is_synthetic_anomaly"] = False
df.loc[anomaly_indices, "is_synthetic_anomaly"] = True

# 6) Quick peek
df.head(), df["is_synthetic_anomaly"].value_counts()


(  transaction_id                  date_time   amount currency  \
 0           tx_1 2025-06-05 07:17:58.684456  -908.64      EUR   
 1           tx_2 2025-10-06 09:17:58.684456   872.15      GBP   
 2           tx_3 2025-09-14 08:17:58.684456  1610.79      GBP   
 3           tx_4 2025-08-06 03:17:58.684456  1651.69      EUR   
 4           tx_5 2025-08-05 00:17:58.684456   839.21      EUR   
 
            debit_account        credit_account  user_id             vendor  \
 0  6300-ProfessionalFees  2100-AccruedExpenses   user_4  Umbrella Services   
 1            6100-Travel  2200-DeferredRevenue   user_5          Globex IT   
 2                6200-IT             1000-Cash  user_15     Stark Supplies   
 3          6500-Salaries               2000-AP   user_5     Stark Supplies   
 4            6100-Travel  2200-DeferredRevenue   user_2     Stark Supplies   
 
   transaction_type  is_synthetic_anomaly  
 0          payment                 False  
 1       adjustment                 Fa

**Step 5 — Save the Synthetic Dataset**

This step writes the synthetic transaction data to a CSV file so it can be reloaded later in the notebook or shared externally.

What this does:

- Saves the entire df DataFrame (including synthetic anomalies and all transaction fields)
to a file named transactions.csv in the Colab working directory.
- Removes the index column for a clean CSV.
- Prints a confirmation message with the number of rows saved.

Why we do this:

- Makes the dataset persistent for the rest of the notebook.
- Allows us to download the file for inspection.
- Ensures the next steps (feature engineering, anomaly detection) start from a clean load, simulating a real-world ingestion pipeline.

In [78]:
df.to_csv("transactions.csv", index=False)
print("Saved synthetic data to transactions.csv with", len(df), "rows.")

Saved synthetic data to transactions.csv with 5000 rows.


**Step 6 — Load the Transaction Dataset**

This step loads the saved synthetic transaction data back into the notebook from the CSV file.

What this does:

- Reads transactions.csv into a pandas DataFrame called df.
- Parses the date_time column as a proper datetime object so we can extract hours, weekdays, months, etc. later.
- Ensures the rest of the pipeline starts from a clean, reproducible dataset—simulating a real ingestion step.

Why this step matters:

- Keeps the workflow consistent with how real production pipelines work (load → process → analyze).
- Ensures that any accidental in-memory modifications from earlier cells don’t leak into the anomaly detection steps.
- Makes it easy for others to run the notebook independently—everything is loaded from the saved CSV.

In [79]:
df = pd.read_csv("transactions.csv", parse_dates=["date_time"])

**Step 7 — Add Statistical Context (Per-Account & Per-Vendor Baselines)**

This step enriches the dataset with statistical baselines that show what “normal” behaviour looks like for each account and vendor.
These baselines provide ground truth context for the LLM so its explanations are factual rather than guessed.

What this code does

1. Create base_df
   -  A copy of the dataset for feature engineering and modelling.
2. Compute per-account statistics
   - For each debit_account, we calculate:
     - acct_avg_amount – average amount
     - acct_median_amount – median amount
     - acct_std_amount – standard deviation
     - acct_tx_count – number of historical transactions
   - These reflect typical behaviour per account.

3. Compute per-vendor statistics
   - Same logic as above, but grouped by vendor:
     - vendor_avg_amount
     - vendor_median_amount
     - vendor_std_amount
     - vendor_tx_count

   - This shows whether a particular vendor is being used unusually.

4. Merge the stats into the main dataset

Each row now has its account/vendor baselines included.

5. Calculate a Z-score vs account

This shows how extreme the amount is compared to the account’s historical norm:

```
z = (amount - account average) / account standard deviation
```

  - High positive z → bigger than typical
  - High negative z → unusually small or negative
  - A great signal for the LLM explanations

6. Clean up invalid Z-scores

Prevents inf or NaN values when account std is 0.

Why this step matters

- IsolationForest learns structure, but it does not explain it.
- The LLM explanations need concrete numbers to avoid hallucinations.
- These statistical fields give the LLM real context like:

> This amount is 3.8× higher than the typical average for account 6100-Travel.

This creates explanations that are accurate, specific, and trustworthy.

In [80]:
# Use the same df you used for IsolationForest training (e.g. df_model or df)
base_df = df.copy()

# Make sure amount is numeric
base_df["amount"] = base_df["amount"].astype(float)

# 1) Stats by debit_account
acct_stats = (
    base_df.groupby("debit_account")["amount"]
    .agg(
        acct_avg_amount="mean",
        acct_median_amount="median",
        acct_std_amount="std",
        acct_tx_count="count",
    )
    .reset_index()
)

# 2) Stats by vendor
vendor_stats = (
    base_df.groupby("vendor")["amount"]
    .agg(
        vendor_avg_amount="mean",
        vendor_median_amount="median",
        vendor_std_amount="std",
        vendor_tx_count="count",
    )
    .reset_index()
)

# 3) Merge these stats back into base_df
base_df = base_df.merge(acct_stats, on="debit_account", how="left")
base_df = base_df.merge(vendor_stats, on="vendor", how="left")

# If you want an amount z-score vs account
base_df["acct_amount_zscore"] = (
    (base_df["amount"] - base_df["acct_avg_amount"]) / base_df["acct_std_amount"]
)

# Replace inf/nan z-scores where std is 0 or missing
base_df["acct_amount_zscore"] = base_df["acct_amount_zscore"].replace([np.inf, -np.inf], np.nan)


**Step 8 — Create Model Features (Time Fields + Log Amount) and Build `df_model`**

This step prepares the features that the IsolationForest model will learn from.
We derive additional time-based fields, transform amounts, select categorical and numeric features, and build the modelling dataset.

What this code does:

1. Extract time-based behavioural features
   - `hour` – posting time of day
   - `dayofweek` – Monday=0 … Sunday=6
   - `month` – captures seasonal or monthly patterns

These help detect unusual posting behaviour (e.g., late-night or weekend transactions).

2. Transform transaction amounts

    `log_amount = log(1 + abs(amount))`

   - Reduces impact of large outliers
   - Makes the distribution more normal
   - Helps IsolationForest detect subtle deviations

3. Define numeric and categorical feature groups

   - Numeric: log_amount, hour, dayofweek, month
   - Categorical: accounts, user IDs, transaction type
(These will later be one-hot encoded.)

4. Build the model-ready dataset

`df_model` includes:

   - All modelling features
   - The statistical context added in Step 7
   - Only rows where all required modelling features are present

This is what we feed into the preprocessing pipeline and IsolationForest model.

What can be tweaked?

- Add new time features (e.g., quarter, week number).
- Add amount-based features (e.g., rolling averages).
- Add more categorical fields (e.g., cost center, region).
- Drop features if model behaves oddly.
- Change feature_cols to test different modelling setups.

In [81]:
# --- Time & amount features on base_df (NOT df) ---
base_df["hour"] = base_df["date_time"].dt.hour
base_df["dayofweek"] = base_df["date_time"].dt.dayofweek
base_df["month"] = base_df["date_time"].dt.month
base_df["log_amount"] = np.log1p(base_df["amount"].abs())
base_df["raw_amount_abs"] = base_df["amount"].abs()

# Features we feed into IsolationForest
num_features = ["log_amount", "raw_amount_abs", "hour", "dayofweek", "month"]
cat_features = ["debit_account", "credit_account", "user_id", "transaction_type"]

feature_cols = num_features + cat_features

# df_model now comes from base_df so it includes the stats columns too
df_model = base_df.dropna(subset=feature_cols).copy()

**Step 9 — Build the ML Pipeline, Train IsolationForest, and Identify Anomalies**

This step constructs a full preprocessing + modelling pipeline, trains the IsolationForest model, and computes anomaly scores for every transaction.

What this does:

- Preprocesses features
  - Numeric fields pass through
  - Categorical fields are one-hot encoded
- Creates and trains an IsolationForest model
  - Learns what “normal” transactions look like
  - Flags roughly 1% as anomalies (contamination=0.01)
- Generates anomaly scores
  - More negative → more unusual
  - `is_anomaly` = -1 marks suspicious rows
- Builds `df_anomalies`, sorted from most to least anomalous.

What you can tweak:

- `contamination`=`0.01` → controls how sensitive you want the system to be
   - 0.005 = stricter
   - 0.02 = more anomalies flagged
- `n_estimators` → more trees = more stable scoring
- Add or remove features to see how model behaviour changes


In [82]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ]
)

iso_forest = IsolationForest(
    n_estimators=200,
    contamination=0.02,  # assume ~1% anomalies
    random_state=42,
)

pipe = Pipeline(steps=[
    ("prep", preprocessor),
    ("model", iso_forest),
])

pipe.fit(df_model[feature_cols])

# Scores & predictions
X_transformed = pipe["prep"].transform(df_model[feature_cols])
df_model["anomaly_score"] = pipe["model"].decision_function(X_transformed)
df_model["is_anomaly"] = pipe["model"].predict(X_transformed)  # -1 anomaly, 1 normal

df_anomalies = df_model[df_model["is_anomaly"] == -1].copy()
df_anomalies = df_anomalies.sort_values("anomaly_score")  # more negative = more anomalous

print("Anomalies found:", (df_model["is_anomaly"] == -1).sum())


Anomalies found: 100


**Step 10 — Select the Top Anomalies for LLM Explanation**

This step chooses the most suspicious transactions to send to the LLM for interpretation.

What this does:

- Takes the first N rows from df_anomalies, which are already sorted by anomaly score.
- These represent the most unusual transactions in the dataset.
- `df_top` will be passed to the OpenAI model to generate human-friendly explanations.

What you can tweak:

- Change TOP_N to adjust how many anomalies you want explained:
  - TOP_N = 5 → faster, good for testing
  - TOP_N = 20 or 50 → richer analysis for stakeholders
- Increase it after you're happy with the model behaviour.

Be mindful that you'll burn through your OpenAI credits

In [83]:
TOP_N = 10
df_top = df_anomalies.head(TOP_N)

**Step 11 — Define the LLM Prompt and Create a Function to Explain Anomalies**

This step sets up the system prompt for the LLM and defines a reusable function that sends one anomalous transaction to OpenAI and returns a structured JSON explanation.

What this does:

1. Defines the system prompt

   The SYSTEM_PROMPT tells the model exactly how to behave:

    - It receives a single transaction and optional statistical context.
    - It must return:
      - severity
      - category
      - short explanation
      - detailed explanation
      - suggested action

    - It must use the statistical values when available (e.g., averages, medians, z-scores).
    - It must not invent numbers.
    - It must reply with valid JSON only.

This ensures clear, consistent, audit-friendly output.

2. Builds a clean transaction payload

   The function extracts relevant fields from each row:

    - Transaction metadata (amount, accounts, date, vendor, etc.)
    - Anomaly score
    - Statistical baselines (avg, median, tx counts, z-scores)

If any stat is missing, it is passed as null, and the model is instructed to handle that case gracefully.

3. Sends the request to OpenAI
   - Uses OpenAI’s structured output mode so the response is guaranteed to be JSON.
   - Ensures the model behaves predictably across all anomalies.

4. Parses the model output

   - If valid JSON → return it.
   - If anything goes wrong → return a fallback explanation so the pipeline never breaks.

**Why this step matters**

This function connects the ML anomaly score with human-readable insights:

   - ML identifies what’s unusual.
   - LLM explains why it’s unusual, using the statistical context you computed.

This pairing is what makes the PoC feel intelligent and usable to finance teams.

**What you can tweak:**

   - Add/remove fields from the transaction payload.
   - Modify severity logic instructions.
   - Add more categories (e.g., vendor anomaly, duplicate risk).
   - Adjust the tone of the explanations.
   - Switch to a different OpenAI model by changing model=MODEL_NAME.

In [84]:
import json

SYSTEM_PROMPT = """
You are an assistant for a subledger accounting platform.

You receive:
- A single transaction with fields like amount, accounts, user, date.
- An anomaly score from an IsolationForest model (more negative = more anomalous).
- Optional statistical context about typical amounts for this account and vendor.

The statistical fields (when present) are:
- acct_avg_amount: average amount for this debit_account.
- acct_median_amount: median amount for this debit_account.
- acct_tx_count: number of historical transactions for this debit_account.
- vendor_avg_amount: average amount for this vendor.
- vendor_median_amount: median amount for this vendor.
- vendor_tx_count: number of historical transactions for this vendor.
- acct_amount_zscore: (amount - acct_avg_amount) / acct_std_amount.

Your job:
1. Decide severity: "low", "medium", or "high".
2. Choose category from: ["amount_anomaly", "timing_anomaly", "account_combo_anomaly", "user_behavior_anomaly", "other"].
3. Provide a short, one-sentence explanation appropriate for a dashboard.
4. Provide a detailed explanation (2–5 sentences) with specific numbers.
5. Suggest a concrete next action for a finance user.

Rules:
- Use the statistical context when available. For example:
  "This amount is about 4.5x the average for this account (X vs typical Y)."
- Do NOT claim that you know "typical" amounts unless the statistical fields are present.
- If the statistical fields are missing or null, speak more generally:
  e.g. "The anomaly score indicates this transaction is unusual, but the exact deviation from normal is not provided."
- Do not invent numbers.
- Keep explanations factual and concise.
- Return valid JSON ONLY with keys:
  ["severity", "category", "short_explanation", "detailed_explanation", "suggested_action"].
"""

def call_openai_for_anomaly(row, client, model=MODEL_NAME):

    """Call OpenAI to explain a single anomalous transaction."""

    # Safe date handling
    date_time = row["date_time"]
    if hasattr(date_time, "isoformat"):
        date_time_str = date_time.isoformat()
    else:
        date_time_str = str(date_time)

    tx = {
        "transaction_id": str(row["transaction_id"]),
        "amount": float(row["amount"]),
        "currency": row.get("currency"),
        "date_time": row["date_time"].isoformat(),
        "debit_account": row.get("debit_account"),
        "credit_account": row.get("credit_account"),
        "user_id": row.get("user_id"),
        "vendor": row.get("vendor"),
        "transaction_type": row.get("transaction_type"),
        "anomaly_score": float(row["anomaly_score"]),
        # statistical context (may be NaN → will serialise as null)
        "acct_avg_amount": row.get("acct_avg_amount"),
        "acct_median_amount": row.get("acct_median_amount"),
        "acct_tx_count": row.get("acct_tx_count"),
        "vendor_avg_amount": row.get("vendor_avg_amount"),
        "vendor_median_amount": row.get("vendor_median_amount"),
        "vendor_tx_count": row.get("vendor_tx_count"),
        "acct_amount_zscore": row.get("acct_amount_zscore"),
    }

    user_prompt = f"""

    Here is an anomalous transaction from an accounting subledger system, including optional statistical context:

    ```json
    {json.dumps(tx, indent=2)}
    Follow the rules and output the JSON response only.
    ```
    """

    # Use JSON response format so we reliably get JSON back
    completion = client.chat.completions.create(
        model=model,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
    )

    text = completion.choices[0].message.content

    try:
        return json.loads(text)
    except json.JSONDecodeError:
        # Fallback if something weird happens
        return {
            "severity": "unknown",
            "category": "other",
            "short_explanation": "Model returned invalid JSON.",
            "detailed_explanation": text,
            "suggested_action": "Review this transaction manually.",
        }

**Step 12 — Generate LLM Explanations and Build the Final Enriched Output**

This step loops through the top anomalous transactions, sends each one to the LLM for interpretation, and combines everything into a final, enriched dataset.

What this does:

1. Calls the LLM for each anomaly
   - Iterates over the most suspicious transactions (`df_top`).
   - For each one, calls the OpenAI function from Step 11.
   - Stores each JSON explanation in a list.

2. Combines original data + LLM output

   Merges:
   - the original anomaly data (`df_top`)
   - the LLM-generated fields (severity, category, explanations, etc.)

   into one unified DataFrame.

3. Displays the enriched dashboard-ready output

   The preview includes:

    - key transaction fields
    - statistical context
    - ML anomaly score
    - LLM severity + category
    - LLM short explanation
    - Suggested next action

   This is the final PoC output you would surface to a finance user or integrate into a UI.

What you can tweak:
   - Add more fields to the preview (e.g., z-scores, vendor stats).
   - Increase or decrease the number of rows displayed.
   - Capture model reasoning for auditing (LLM “thoughts” not included by default).
   - Add IDs or correlation IDs for tracking.

In [85]:
explanations = []
for _, row in df_top.iterrows():
    explanations.append(call_openai_for_anomaly(row, client))

df_enriched = pd.concat(
    [df_top.reset_index(drop=True), pd.DataFrame(explanations)],
    axis=1
)

df_enriched[[
    "transaction_id", "date_time", "amount", "debit_account", "credit_account", "vendor",
    "acct_avg_amount", "vendor_avg_amount", "anomaly_score",
    "severity", "category", "short_explanation", "detailed_explanation", "suggested_action"
]].head(10)


,transaction_id,date_time,amount,debit_account,credit_account,vendor,acct_avg_amount,vendor_avg_amount,anomaly_score,severity,category,short_explanation,detailed_explanation,suggested_action
0,tx_465,2025-05-20 23:17:58.684456,177103.00,6400-Rent,2000-AP,Wayne Travel,1637.711580,1790.871535,-0.035896,high,amount_anomaly,This rent payment amount is highly anomalous c...,"The transaction amount of $177,103 is about 10...",Review this transaction in detail to verify it...
1,tx_1140,2025-08-24 22:17:58.684456,-3029.55,6100-Travel,2200-DeferredRevenue,ACME Events,957.658634,809.589852,-0.024372,low,amount_anomaly,The transaction amount is somewhat lower than ...,"The transaction amount is -3029.55 GBP, which ...",Review the nature of this adjustment transacti...
2,tx_4856,2025-09-20 23:17:58.684456,812.88,6500-Salaries,2200-DeferredRevenue,Initech Consulting,804.215667,1367.780524,-0.020313,low,amount_anomaly,This transaction amount closely aligns with ty...,The transaction amount of 812.88 EUR is very c...,Review transaction details briefly to confirm ...
3,tx_623,2025-06-17 22:17:58.684456,3097.52,6300-ProfessionalFees,1000-Cash,Globex IT,1237.957393,805.650876,-0.018653,low,amount_anomaly,This transaction amount is somewhat higher tha...,The transaction amount of 3097.52 EUR is about...,Review the transaction details for justificati...
4,tx_4291,2025-06-01 02:17:58.684456,2677.30,6200-IT,2100-AccruedExpenses,Initech Consulting,1030.313540,1367.780524,-0.017552,low,amount_anomaly,Transaction amount is moderately higher than t...,This transaction amount of 2677.3 EUR is about...,Review the transaction details to confirm appr...
5,tx_3562,2025-06-18 23:17:58.684456,538.69,6200-IT,2000-AP,Umbrella Services,1030.313540,829.720766,-0.017024,low,amount_anomaly,Transaction amount is slightly below typical a...,This transaction amount of 538.69 EUR is below...,Review this transaction to confirm the lower a...
6,tx_1216,2025-06-08 23:17:58.684456,1341.07,6000-Marketing,2000-AP,ACME Events,1101.998086,809.589852,-0.016333,low,amount_anomaly,Transaction amount is slightly above average b...,The transaction amount of 1341.07 EUR is about...,Review the transaction details to confirm legi...
7,tx_3919,2025-05-30 03:17:58.684456,150382.61,6200-IT,1000-Cash,Wayne Travel,1030.313540,1790.871535,-0.015366,high,amount_anomaly,This transaction amount is extremely high comp...,"The transaction amount of EUR 150,382.61 is ab...",Review the transaction details and supporting ...
8,tx_362,2025-10-26 22:17:58.684456,-420.83,6200-IT,2000-AP,Globex IT,1030.313540,805.650876,-0.014981,low,amount_anomaly,This transaction amount is slightly lower than...,"The transaction amount is -420.83 EUR, which i...",Review the transaction details to confirm the ...
9,tx_777,2025-08-03 23:17:58.684456,-477.83,6100-Travel,2100-AccruedExpenses,Initech Consulting,957.658634,1367.780524,-0.014658,low,amount_anomaly,The transaction amount is negative and slightl...,"This transaction has an amount of -477.83 EUR,...",Review the transaction details to confirm that...


**Step 13 — Export the Dataset to CSV**

This step saves the full dataset dataset—including ML scores, statistical context, and LLM explanations—to a CSV file for download or further analysis.

What this does:

- Creates a copy of the dataframe model and adds additional columns that are expected from the LLM.
- Iterates through the anomaly dataset and updates the relevant fields where a match can be found.
- Produces a final deliverable you can:
  - download from Colab

This completes the end-to-end PoC pipeline.

In [86]:

# Start with a full copy of the dataset used by the model
df_full_export = df_model.copy()

# Add empty columns for LLM outputs
for col in ["severity", "category", "short_explanation", "detailed_explanation", "suggested_action"]:
    df_full_export[col] = None

# Insert LLM explanations only for the anomalies we explained (df_top)
for idx, row in df_enriched.iterrows():   # df_enriched matches df_top row order
    original_idx = df_top.index[idx]      # get the original row index in df_model
    for col in ["severity", "category", "short_explanation", "detailed_explanation", "suggested_action"]:
        df_full_export.loc[original_idx, col] = row[col]

# Save the complete dataset
df_full_export.to_csv("full_dataset_with_selected_llm_explanations.csv", index=False)

print("Full dataset saved with LLM explanations only for selected anomalies.")
# df_enriched.to_csv("enriched_anomalies_openai.csv", index=False)


Full dataset saved with LLM explanations only for selected anomalies.
